# EDA: Berita FAKE vs REAL 

Notebook ini adalah panduan lengkap untuk melakukan Data Profiling dan Analisis Data Eksplorasi (EDA) pada dataset berita `fake_or_real_news.csv`. Tujuannya adalah untuk memahami karakteristik data teks secara mendalam sebelum masuk ke tahap pemodelan.

Link : https://www.kaggle.com/datasets/jillanisofttech/fake-or-real-news/data

## 1. Data Acquisition

### 1.1 Import Library

In [ ]:
# Library untuk manipulasi data
import pandas as pd
import numpy as np

# Library untuk visualisasi
import seaborn as sns
import matplotlib.pyplot as plt

# Mengatur agar plot ditampilkan di dalam notebook
%matplotlib inline

print("Library siap digunakan!")

### 1.2 Memuat Dataset

In [ ]:
# Membaca file CSV ke dalam DataFrame
df = pd.read_csv('data/text_real_or_fake_news.csv')

# Menampilkan 5 baris pertama data
print("5 baris pertama dari dataset:")
df.head()

## 2. Data Profiling / Exploratory Data Analysis (EDA)

### 2.1 Informasi Dasar DataFrame

In [ ]:
# Menampilkan informasi ringkas (tipe data, memori, dll.)
print("Informasi Umum Dataset:")
df.info()

print("\nStatistik Deskriptif:")

# Memeriksa jumlah nilai yang hilang (missing values) per kolom
print("Jumlah Nilai yang Hilang per Kolom:")
print(df.isnull().sum())

### 2.2 Analisis Distribusi Label

In [ ]:
# Visualisasi distribusi label
plt.figure(figsize=(8, 6))
sns.countplot(x='label', data=df, palette='viridis')
plt.title('Distribusi Berita FAKE vs REAL', fontsize=16)
plt.xlabel('Label', fontsize=12)
plt.ylabel('Jumlah', fontsize=12)
plt.show()

# Menampilkan jumlah pastinya
print(df['label'].value_counts())

### 2.3 Analisis Karakteristik Teks

In [ ]:
# Menghitung jumlah kata dalam setiap teks
df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))

# Menghitung jumlah karakter dalam setiap teks
df['char_count'] = df['text'].apply(lambda x: len(str(x)))

# Menghitung panjang rata-rata kata
def avg_word_length(text):
    words = str(text).split()
    if len(words) == 0:
        return 0
    return sum(len(word) for word in words) / len(words)

df['avg_word_length'] = df['text'].apply(avg_word_length)

print("Fitur baru berhasil ditambahkan (word_count, char_count, avg_word_length):")
df[['label', 'word_count', 'char_count', 'avg_word_length']].head()

In [ ]:
# Mengatur ukuran plot
plt.figure(figsize=(18, 12))

# Plot 1: Distribusi Jumlah Kata
plt.subplot(2, 2, 1)
sns.histplot(data=df, x='word_count', hue='label', multiple='stack', bins=50)
plt.title('Distribusi Jumlah Kata')

# Plot 2: Distribusi Jumlah Karakter
plt.subplot(2, 2, 2)
sns.histplot(data=df, x='char_count', hue='label', multiple='stack', bins=50)
plt.title('Distribusi Jumlah Karakter')

# Plot 3: Boxplot Panjang Rata-rata Kata
plt.subplot(2, 2, 3)
sns.boxplot(data=df, x='label', y='avg_word_length')
plt.title('Panjang Rata-rata Kata')

plt.tight_layout()
plt.show()

### 2.4 Analisis N-grams (Kata yang Sering Muncul)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Fungsi untuk membuat plot kata yang paling umum
def plot_top_ngrams(corpus, title, n=20, ngram_range=(1,1)):
    vec = CountVectorizer(stop_words='english', ngram_range=ngram_range).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    
    top_words_df = pd.DataFrame(words_freq[:n], columns=['Kata/Frasa', 'Frekuensi'])
    
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Frekuensi', y='Kata/Frasa', data=top_words_df, palette='plasma')
    plt.title(title, fontsize=16)
    plt.show()

# Pisahkan teks berita FAKE dan REAL
fake_news_text = df[df['label'] == 'FAKE']['text']
real_news_text = df[df['label'] == 'REAL']['text']

# Plot 20 kata tunggal (unigram) yang paling umum di berita FAKE
plot_top_ngrams(fake_news_text, 'Top 20 Kata Tunggal di Berita FAKE', ngram_range=(1,1))

# Plot 20 kata tunggal (unigram) yang paling umum di berita REAL
plot_top_ngrams(real_news_text, 'Top 20 Kata Tunggal di Berita REAL', ngram_range=(1,1))

## Kesimpulan Analisis

Dari Data Profiling yang mendalam ini, kita mendapatkan pemahaman yang lebih baik tentang dataset. Kita tidak hanya tahu bahwa datanya seimbang, tetapi juga menemukan fitur-fitur potensial seperti:
1.  **Panjang Teks**: Berita REAL cenderung lebih panjang.
2.  **Kata Kunci (N-grams)**: Ada perbedaan kosakata yang signifikan antara kedua jenis berita.

Informasi ini sangat berharga dan akan membantu model *machine learning* untuk belajar membedakan kedua kelas dengan lebih baik. Anda sekarang siap untuk melanjutkan ke tahap **Data Preprocessing** dan **Modeling**.